# Import Libraries

In [1]:
import matplotlib
matplotlib.use('tkAgg')
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2
import keras
import glob
import os
from time import time
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import pickle
from keras.callbacks import TensorBoard
from sklearn.model_selection import StratifiedShuffleSplit

Using TensorFlow backend.


# Configure Parameters

In [2]:
cwd = os.getcwd()
logsdir = os.path.join(cwd,"logs10")
batchsize = 8
number_class = 7
epochs = 10
ratio_testset = 0.15
lrate = 0.0001
numbatchsplit = 10

# Load dataset

In [3]:

#------------First time------------
y1 = len(glob.glob(os.path.join("Dataset\\Image100_resize" ,'*jpg')))
y2 = len(glob.glob(os.path.join("Dataset\\Image110_resize" ,'*jpg')))
y3 = len(glob.glob(os.path.join("Dataset\\Image120_resize" ,'*jpg')))
y4 = len(glob.glob(os.path.join("Dataset\\Image130_resize" ,'*jpg')))
y5 = len(glob.glob(os.path.join("Dataset\\Image140_resize" ,'*jpg')))
y6 = len(glob.glob(os.path.join("Dataset\\Image150_resize" ,'*jpg')))
y7 = len(glob.glob(os.path.join("Dataset\\Image160_resize" ,'*jpg')))

# X_data
X = []
count = 0
for i in glob.glob(os.path.join("Dataset\\Image100_resize" ,'*jpg')): #100
	img = cv2.imread(i)	
	if count == 0:
		X = img 
		count = 1
	else:
		X = np.append(X,img, axis = 0)	
print("Loading 10%..")
for i in glob.glob(os.path.join("Dataset\\Image110_resize" ,'*jpg')): #110
	img = cv2.imread(i)	
	#img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
	X = np.append(X,img, axis = 0)
print("Loading 30%..")
for i in glob.glob(os.path.join("Dataset\\Image120_resize" ,'*jpg')): #120
	img = cv2.imread(i)	
	#img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
	X = np.append(X,img, axis = 0)
print("Loading 50%..")
for i in glob.glob(os.path.join("Dataset\\Image130_resize" ,'*jpg')): #130
	img = cv2.imread(i)	
	#img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
	X = np.append(X,img, axis = 0)
print("Loading 70%..")
for i in glob.glob(os.path.join("Dataset\\Image140_resize" ,'*jpg')): #140
	img = cv2.imread(i)	
	#img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
	X = np.append(X,img, axis = 0)
for i in glob.glob(os.path.join("Dataset\\Image150_resize" ,'*jpg')): #150
	img = cv2.imread(i)	
	#img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
	X = np.append(X,img, axis = 0)
print("Loading 90%..")
for i in glob.glob(os.path.join("Dataset\\Image160_resize" ,'*jpg')): #160
	img = cv2.imread(i)	
	#img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
	X = np.append(X,img, axis = 0)
print("Finish")
X = X.reshape(-1,66,200,3)
X.astype('float32')
#y data
y = np.asarray([0]*y1 + [1]*y2 + [2]*y3 + [3]*y4 + [4]*y5 + [5]*y6 + [6]*y7)
y.astype('uint8')

with open("data.txt", "wb") as data:
    pickle.dump((X,y), data)
    

Loading 10%..
Loading 30%..
Loading 50%..
Loading 70%..
Loading 90%..
Finish


In [3]:
#------------------from the second time onwards-----------------
with open("data.txt", "rb") as data:
    X,y = pickle.load(data)

# Initialize model

In [4]:
model = Sequential()
model.add(Conv2D(8, kernel_size=(7, 7),				 
                 activation='relu',
				 strides=(2, 2),
                 input_shape= (66,200,3),
				 padding='valid',))
model.add(Conv2D(16, kernel_size=(5, 5), 
				 activation='relu',
				 strides=(2, 2),
				 padding='valid',))
model.add(Conv2D(32, kernel_size=(5, 5), 
				 activation='relu',
				 strides=(2, 2),
				 padding='valid',))
model.add(Conv2D(64, kernel_size=(3, 3), 
				 activation='relu',
				 strides=(1, 1),
				 padding='valid',))
model.add(Conv2D(64, kernel_size=(3, 3), 
				 activation='relu',
				 strides=(1, 1),
				 padding='valid',))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(number_class, activation='softmax'))
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 97, 8)         1184      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 47, 16)        3216      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 32)         12832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 20, 64)         18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dens

# Split dataset : There are two methods to split data: Train/Test and K-Fold

In [5]:
#--------------------Train/Test-----------------------
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=ratio_testset, random_state=1)
y_train = np_utils.to_categorical(y_train, number_class)
y_valid = np_utils.to_categorical(y_valid, number_class)
print("Splitting dataset follows as Train/Test method")

Splitting dataset follows as Train/Test method


In [7]:
#--------------------K-Fold---------------------------
y = np_utils.to_categorical(y, number_class)
kfold = StratifiedShuffleSplit(n_splits=numbatchsplit, random_state=7,test_size=ratio_testset)
print("Splitting dataset follows as k-fold method")

Splitting dataset follows as k-fold method


# Training model

In [ ]:
#--------------------Train with splitting data follows as Train/Test method------------
checkpoint_path = os.path.join(logsdir, "{epoch:04d}.h5")
tensorboard = [TensorBoard(log_dir=logsdir,
							histogram_freq=0, write_graph=True, write_images=False),
							keras.callbacks.ModelCheckpoint(checkpoint_path,
							verbose=0, save_best_only=True),]
Adam = keras.optimizers.Adam(lr=lrate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

model.fit(X_train, y_train,
					batch_size=batchsize,
					epochs=epochs,
					shuffle=True,
					validation_data=(X_valid, y_valid),
					callbacks=tensorboard)

Instructions for updating:
Use tf.cast instead.
Train on 16636 samples, validate on 2936 samples
Epoch 1/10
 4608/16636 [=======>......................] - ETA: 19s - loss: 1.3287 - acc: 0.4894

In [8]:
#--------------------Train with splitting data follows as K-fold method------------
Adam = keras.optimizers.Adam(lr=lrate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

i = 0
for train_index, test_index in kfold.split(X,y):
    checkpoint_path = os.path.join(logsdir,"Model_fold_" + str(i) + "_Epoch_{epoch:04d}.h5")
    tensorboard = [TensorBoard(log_dir=logsdir,
 					histogram_freq=0, write_graph=True, write_images=False),
					keras.callbacks.ModelCheckpoint(checkpoint_path,
 					verbose=0, save_best_only=True),]
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train,
                        batch_size=batchsize,
                        epochs=epochs,
                        shuffle=True,
                        validation_data=(X_test, y_test),
                        callbacks=tensorboard)
    i = i +1


Train on 14749 samples, validate on 2603 samples
Epoch 1/10
14749/14749 [==============================] - 14s 938us/step - loss: 0.3385 - acc: 0.8630 - val_loss: 0.3439 - val_acc: 0.8636
Epoch 2/10
 5344/14749 [=========>....................] - ETA: 8s - loss: 0.3014 - acc: 0.8795

KeyboardInterrupt: 